In [1]:
import csv
import math

import matplotlib.pyplot as plt
import numpy as np

from keras import applications
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model, load_model
from keras.layers import Dropout, Flatten, Dense, Input, Conv2D, MaxPooling2D
from keras.initializers import glorot_uniform
from keras.applications.vgg16 import preprocess_input

from sklearn.metrics import confusion_matrix, classification_report

Using TensorFlow backend.


In [2]:
# read the CSV into memory
prices = []
image_paths = []

data_path = "../datasets/bikes_im/"
with open("bikes_classified.csv") as file:
    reader = csv.reader(file)
    i = -1
    for row in reader:
        i += 1
        index = row[0]
        name = row[1]
        msrp = row[2]
        label = row[3]
        
        image_path = data_path + index + '.jpg'
        image_paths.append(image_path)
        prices.append(str(label))

train_indices = np.load("bikes_train_indices.npy")
test_indices = np.load("bikes_test_indices.npy")
print(train_indices.shape)
print(test_indices.shape)

(19658,)
(2185,)


In [3]:
def image_generator(indices, batch_size):

    num_batches = int(len(indices) / batch_size)
    
    while True:
        for batch_i in range(num_batches):
            if batch_i == num_batches - 1:
                # special case: return as many as possible
                start_i = batch_i * batch_size
                batch_indices = indices[start_i:]
                
                X = np.zeros((len(batch_indices), 224, 224, 3))
                Y = np.zeros((len(batch_indices), 4))
            
            else:
                start_i = batch_i * batch_size
                end_i = start_i + batch_size

                batch_indices = indices[start_i:end_i]

                X = np.zeros((batch_size, 224, 224, 3))
                Y = np.zeros((batch_size, 4))
            
            for i, index in enumerate(batch_indices):
                img = image.load_img(image_paths[index], target_size=(224, 224))
                X[i, :, :, :] = image.img_to_array(img)                
                p = prices[index]
                if p == "25":
                    Y[i,:] = np.array([1,0,0,0])
                if p == "50":
                    Y[i,:] = np.array([0,1,0,0])
                if p == "75":
                    Y[i,:] = np.array([0,0,1,0])
                if p == "100":
                    Y[i,:] = np.array([0,0,0,1])
            
            # use vgg16 preprocessing
            X = preprocess_input(X)
            
            yield (X, Y)

In [4]:
# Hyperparameters

num_settings = 1

hp_dropout = [0.2] * num_settings

#RMSprop
hp_lr = [0.0001] * num_settings
hp_rho = [0.9] * num_settings
hp_epsilon = [1e-07] * num_settings
hp_decay = [0.0] * num_settings

# Number of hidden units
hp_hidden = [300] * num_settings

# Minibatch size
hp_mbsize = [64] * num_settings
minibatch_size = hp_mbsize[0]

train_steps = math.ceil(len(train_indices) / minibatch_size)
test_steps = math.ceil(len(test_indices) / minibatch_size)

num_epochs = 20

In [6]:
# store the results of each setting
train_losses = np.zeros(num_settings)
dev_losses = np.zeros(num_settings)

for setting in range(num_settings):

    img_input = Input(shape=(224, 224, 3))

    # block 1
    x = Conv2D(56, (5, 5), activation='relu', padding='same', kernel_initializer='glorot_normal', name='conv1')(img_input)
    x = Conv2D(56, (3, 3), activation='relu', padding='same', kernel_initializer='glorot_normal', name='intermed')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool1')(x)

    # block 2
    x = Dropout(hp_dropout[setting])(x)
    x = Conv2D(64, (4, 4), activation='relu', padding='same', kernel_initializer='glorot_normal', name='conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool2')(x)

    # block 3
    x = Dropout(hp_dropout[setting])(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='glorot_normal', name='conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool3')(x)

    # block 4
    x = Dropout(hp_dropout[setting])(x)
    x = Conv2D(40, (4, 4), activation='relu', padding='same', kernel_initializer='glorot_normal', name='conv4')(x)
    x = MaxPooling2D((3, 3), strides=(3, 3), name='pool4')(x)

    x = Flatten(name='flatten')(x)
    x = Dropout(hp_dropout[setting])(x)
    x = Dense(256, activation='relu', name='fc1', kernel_initializer='glorot_normal')(x)
    x = Dense(256, activation='relu', name='fc2', kernel_initializer='glorot_normal')(x)
    x = Dense(4, activation='softmax', name='output', kernel_initializer='glorot_normal')(x)
    
    new_model = Model(img_input, x, name='new_network_classification')
    
    print(new_model.summary())
    
    # Adam optimizer
    new_model.compile(loss='categorical_crossentropy',
                      optimizer=optimizers.Adam(
                              lr=hp_lr[setting],
                              decay=hp_decay[setting]),
                      metrics=['accuracy'])
    
    checkpoint_path = '/output/new-network-classification-best.hdf5'
    
    # keep a checkpoint
    checkpoint = ModelCheckpoint(checkpoint_path,
                                monitor='val_acc',
                                save_best_only=True,
                                mode='max')

    minibatch_size = hp_mbsize[setting]

    train_steps = math.ceil(len(train_indices) / minibatch_size)
    test_steps = math.ceil(len(test_indices) / minibatch_size)

    # fine-tune the model
    history = new_model.fit_generator(
        image_generator(train_indices, minibatch_size),
        steps_per_epoch=train_steps,
        epochs=num_epochs,
        validation_data=image_generator(test_indices, minibatch_size),
        nb_val_samples=test_steps,
        callbacks=[checkpoint])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 224, 224, 56)      4256      
_________________________________________________________________
intermed (Conv2D)            (None, 224, 224, 56)      28280     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 56)      0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 112, 112, 56)      0         
_________________________________________________________________
conv2 (Conv2D)               (None, 112, 112, 64)      57408     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 64)        0         
__________

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:66: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:66: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=308, epochs=20, validation_data=<generator..., callbacks=[<keras.ca..., validation_steps=35)`


Epoch 1/20
308/308 [==============================] - 341s - loss: 1.7401 - acc: 0.4140 - val_loss: 0.9668 - val_acc: 0.5767
Epoch 2/20
308/308 [==============================] - 330s - loss: 0.9594 - acc: 0.5612 - val_loss: 0.8697 - val_acc: 0.6189
Epoch 3/20
308/308 [==============================] - 329s - loss: 0.8659 - acc: 0.6122 - val_loss: 0.8040 - val_acc: 0.6487
Epoch 4/20
308/308 [==============================] - 330s - loss: 0.8035 - acc: 0.6437 - val_loss: 0.7604 - val_acc: 0.6740
Epoch 5/20
308/308 [==============================] - 329s - loss: 0.7486 - acc: 0.6657 - val_loss: 0.7302 - val_acc: 0.6865
Epoch 6/20
308/308 [==============================] - 329s - loss: 0.7043 - acc: 0.6920 - val_loss: 0.6847 - val_acc: 0.7132
Epoch 7/20
308/308 [==============================] - 329s - loss: 0.6649 - acc: 0.7090 - val_loss: 0.6675 - val_acc: 0.7123
Epoch 8/20
308/308 [==============================] - 329s - loss: 0.6279 - acc: 0.7276 - val_loss: 0.6492 - val_acc: 0.7203


In [ ]:
# get predictions on each batch yielded the validation generator.

new_model = load_model('/output/new-network-classification-best.hdf5')

In [7]:
true_label = []
predicted_label = []
for index in test_indices:
    msrp = prices[index]
    true_label.append(str(msrp))
    
    path = image_paths[index]
    img = image.load_img(path, target_size=(224, 224))
    data = np.expand_dims(image.img_to_array(img), axis=0)
    data = preprocess_input(data)
    
    # Prediction outputs softmax vector
    prediction = new_model.predict(data)
    
    # Set most confident prediction as label, and convert it to our price scale
    label = np.argmax(prediction) * 25 + 25
    predicted_label.append(str(label))

In [8]:
print("Classification report:\n%s\n"
      % (classification_report(true_label, predicted_label)))
print("Confusion matrix:\n%s" % confusion_matrix(true_label, predicted_label))

Classification report:
             precision    recall  f1-score   support

        100       0.86      0.79      0.83       496
         25       0.92      0.82      0.87       596
         50       0.75      0.80      0.77       563
         75       0.71      0.80      0.75       530

avg / total       0.81      0.80      0.80      2185


Confusion matrix:
[[394   2   2  98]
 [  1 489  97   9]
 [  8  38 448  69]
 [ 53   4  51 422]]
